In [ ]:
# from pyspark.sql import SparkSession
# import os
# os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
# spark = SparkSession.builder.appName("tkt_kwd_model").enableHiveSupport().getOrCreate()

In [ ]:
import sys
sys.path.append("..")
sys.path.append(".")
import numpy as np 
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout

In [2]:
from WDL.model import WideDeep
from WDL.modules import sparseFeature, denseFeature

In [ ]:
# example
# 两个 dense feature
dense_f1 = denseFeature('dense_feature_1')
dense_f2 = denseFeature('dense_feature_2')
# 两个 sparse feature
sparse_f1 = sparseFeature('sparse_feature_1', feat_num=2, embed_dim=8) 
sparse_f2 = sparseFeature('sparse_feature_2', feat_num=5, embed_dim=8)
# 
dense_feature_list = [dense_f1, dense_f2] # 所有连续特征
sparse_feature_list = [sparse_f1, sparse_f2] # 所有离散特征
# 分别指定wide  deep侧特征
wide_feature_list = [dense_f1, dense_f2] # 指定wide侧的特征，不指定则默认所有
deep_feature_list = [dense_f2, sparse_f1] # 指定deep侧的特征，不指定则默认所有


model = WideDeep(dense_feature_list, sparse_feature_list, wide_feature_list, deep_feature_list) 

In [ ]:
model.summary()

In [22]:
# # 模型输入，每个特征一个input组成list
# inputs = [Input(shape=(1,), dtype=tf.float32) for _ in range(len(dense_feature_list) + len(sparse_feature_list))]
# outputs = model(inputs)

In [12]:
batch = 128
d_f1 = tf.constant(np.random.rand(batch, 1))
d_f2 = tf.constant(np.random.rand(batch, 1))
s_f1 = tf.constant(np.random.randint(2, size=(batch, 1)), dtype=tf.float32)
s_f2 = tf.constant(np.random.randint(5, size=(batch, 1)), dtype=tf.float32)
inputs = [d_f1, d_f2, s_f1, s_f2]
outputs = model(inputs)
print(outputs.shape)

(128, 1)
